# Who am I?
## Cesare Placanica

## That was easy... :-)

* Telco engineer @ Red Hat
  * Formerly in Cisco Photonics for many years
* Always worked in Telecommunication industry
  * Which is good since I dig big labs a lot!
* Linux and Python addicted since ever.
* Eventually, I'm trying to be an event organizer for PyMI
  * Pythonistas interviewer!

# Thanks Python Milano for having me!

# Agenda

* Kubernetes Introduction (at fast pace)
* What's a Kubernetes Operator
* Cool tools for Operator development: Kopf and Kwok

# TODO
Add references, books, my videos to really learn kubernetes.

## Streching the Pareto Principle
I can convey the 80% of the content with 20% of time.

OK...

# What Kubernetes is?

Kubernetes is a [Loop](https://fabiensanglard.net/quakeSource/)!

# Are you nuts?

## Let me write it, in Python of course

```python
while True:
    current_state = get_current_state()
    desidered_state = get_desidered_state()
    reconcile(current_state, desidered_state)
```

OK, this is a super high level description to the thing but can help to set the context about what Kubernetes is

Somebody calls the simple loop above a reconciliation loop, a sort of pattern.

This enables you to be "declarative", just state what you need from the system, not *how* reach a "desidered" condition.

And enable Kubernetes to be "Intent based": match the resources with users "desire".

# TODO
Diffence with docker, pet/cattle, lifecycle use cases failover pods, scaling (deployments / replicaset), expose a service (load-balancer, service), upgrades (RollingUpdates) and another gazzilion of things.

* Kubernetes came out many lessons learned by people who actually built a lot of distributed system.
* It's made to build computer clusters and have a standard way to deal with them.
* Has the "single source of truth for the cluster", the etcd database, you can recovery from crashes by doing backups.
* Is API centric.
* **It's declarative!**
* It's extendable using the [Custom Resource Definition](https://kubernetes.io/docs/concepts/extend-kubernetes/api-extension/custom-resources/)
* Even the loop is extendable, using the [Operator Pattern](https://kubernetes.io/docs/concepts/extend-kubernetes/operator/), you can add custom maintenance automation scripts for your application.